In [29]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [30]:
from huggingface_hub import notebook_login
notebook_login()

In [129]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1776


In [130]:
from datasets import Dataset, load_metric

In [131]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [132]:
len(train)

17

In [133]:
train['label'].value_counts()

0    6
2    6
1    5
Name: label, dtype: int64

In [134]:
test['label'].value_counts()

0    597
2    594
1    585
Name: label, dtype: int64

In [135]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [136]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
252,a77138a065b1b594fdb7351f9503be4496995b97,intellij-community,fixed django template commenter again (PY-1949)--,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
295,7ed444678012a695278f54ab86ca4a5dd73537c7,intellij-community,one more test that fails; some more code moved...,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1425,3f3ff95b46cb0ef97704396501b7c05b778cc448,Vala,cairo: Use closure param to ReadFunc and Write...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1275,454302cf65c3c617149f28aee819549dd02f3680,Mylyn Reviews,Move versions-task bridge to reviews\n,2,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1474,74b5e8ad3b43949f122e6430fe576ebdd44cf04a,Vala,vapigen: add support for type_parameters attri...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85,44a27c5cd76f44e435671c69d1d8f60c42a2b420,hbase,HBASE-11920 Add CP hooks for ReplicationEndPoi...,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
545,daab3db062b9b8cddbb653fedca91288cf5a0684,kotlin,Add WITH_RUNTIME and WITH_REFLECT directives t...,2,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1297,464fd741533548c6fec7393299185c293de0d862,Mylyn Reviews,392682: Show details for changesets\n\nUse the...,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
971,8c9507ec33019b88dd43ab47e1bd50d22332740b,Valadoc,girmetadata: allow to override <ONLINE>\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1157,a57bee46d7db4cd6257d45a0733cf575602b0bb3,tapiji,Creates `stable` branch.\n\nThe stable branch ...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [138]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [139]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [140]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [141]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [142]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [143]:
def compute_metrics(eval_pred):
    accuracy = load_metric('accuracy')
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [144]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

<ipython-input-141-ebcd56f8ad98>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.102897,{'precision': 0.317654299680386},{'recall': 0.31475225225225223},{'f1': 0.2808046078779659},{'accuracy': 0.31475225225225223}


Trainer is attempting to log a value of "{'precision': 0.317654299680386}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.31475225225225223}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.2808046078779659}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.31475225225225223}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=2, training_loss=1.1236425638198853, metrics={'train_runtime': 66.9968, 'train_samples_per_second': 0.254, 'train_steps_per_second': 0.03, 'total_flos': 1057078711458.0, 'train_loss': 1.1236425638198853, 'epoch': 1.0})